<a href="https://colab.research.google.com/github/BOOBESH7/BOOBESH7/blob/main/project_SA(main).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [14]:
from google.colab import files
uploaded = files.upload()

Saving all-data.csv to all-data (1).csv


In [15]:

!pip install -q scikit-learn matplotlib seaborn


In [16]:
import pandas as pd
import io

# Load dataset
filename = next(iter(uploaded))  # Gets the uploaded file name
df = pd.read_csv(io.BytesIO(uploaded[filename]), header=None, encoding='latin-1') # Changed encoding to 'latin-1'

# Rename columns for clarity before accessing them
df = df.rename(columns={0: 'sentiment', 1: 'text'})

# Preview the data
print(df.head())
# Example fix in pandas

  sentiment                                               text
0   neutral  According to Gran , the company has no plans t...
1   neutral  Technopolis plans to develop in stages an area...
2  negative  The international electronic industry company ...
3  positive  With the new production plant the company woul...
4  positive  According to the company 's updated strategy f...


In [18]:
# STEP 1: Import Required Libraries
import pandas as pd
import io
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import (
    classification_report, confusion_matrix, accuracy_score,
    f1_score, roc_auc_score, roc_curve
)
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from sklearn.preprocessing import label_binarize

# STEP 2: Load and Prepare Dataset
# The preview shows 'all-data.csv' has no header.
# Column 0 is sentiment, Column 1 is text.
filename = next(iter(uploaded))
df = pd.read_csv(io.BytesIO(uploaded[filename]), header=None, encoding='latin-1')

# Rename columns using position because there is no header in the CSV
df = df.rename(columns={0: 'sentiment', 1: 'text'})

# Clean data
df = df.dropna(subset=['text', 'sentiment'])
df['sentiment'] = df['sentiment'].str.strip().str.lower()

# Get unique classes present in the data
classes = sorted(df['sentiment'].unique().tolist())
print(f"Detected classes: {classes}")

# STEP 3: Train-Test Split
X_train, X_test, y_train, y_test = train_test_split(df['text'], df['sentiment'], test_size=0.2, random_state=42)

# STEP 4: Vectorization
vectorizer = TfidfVectorizer()
X_train_vec = vectorizer.fit_transform(X_train)
X_test_vec = vectorizer.transform(X_test)

# STEP 5: Model Training
model = MultinomialNB()
model.fit(X_train_vec, y_train)

# STEP 6: Predictions
y_pred = model.predict(X_test_vec)
y_proba = model.predict_proba(X_test_vec)

# STEP 7: Evaluation Metrics
print("\nClassification Report:\n")
print(classification_report(y_test, y_pred))

print("\nAccuracy:", accuracy_score(y_test, y_pred))
print("F1 Score (macro):", f1_score(y_test, y_pred, average='macro'))

# Confusion Matrix
conf_matrix = confusion_matrix(y_test, y_pred)
plt.figure(figsize=(8,6))
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues', xticklabels=model.classes_, yticklabels=model.classes_)
plt.title("Confusion Matrix")
plt.xlabel("Predicted")
plt.ylabel("Actual")
plt.show()

# STEP 8: ROC Curve (Multi-Class)
y_test_bin = label_binarize(y_test, classes=model.classes_)
n_classes = len(model.classes_)

fpr = dict()
tpr = dict()
roc_auc = dict()

for i in range(n_classes):
    fpr[i], tpr[i], _ = roc_curve(y_test_bin[:, i], y_proba[:, i])
    roc_auc[i] = roc_auc_score(y_test_bin[:, i], y_proba[:, i])

plt.figure(figsize=(10, 7))
for i in range(n_classes):
    plt.plot(fpr[i], tpr[i], label=f'ROC curve for {model.classes_[i]} (area = {roc_auc[i]:.2f})')

plt.plot([0, 1], [0, 1], 'k--')
plt.title('Multi-Class ROC Curve')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.legend(loc='lower right')
plt.grid()
plt.show()

# STEP 9: Predicted Sentiment Distribution
plt.figure(figsize=(8, 6))
sns.countplot(x=y_pred, hue=y_pred, palette='Set2', legend=False)
plt.title('Predicted Sentiment Distribution')
plt.xlabel('Sentiment')
plt.ylabel('Number of Samples')
plt.show()

KeyError: ['text', 'sentiment']